# **Control del formato de salida**
En el desarrollo de aplicaciones que utilizan modelos de lenguaje, uno de los desafíos clave es garantizar que la salida generada sea estructurada, predecible y fácil de procesar. Para lograrlo, existen técnicas específicas que permiten controlar el formato de salida de manera precisa.

Algunos de ellos son:
* **Function Calling**
* **Outlines**



### **Instalar e importar librerias**

In [ ]:
%pip install openai
%pip install transformers torch
from openai import OpenAI
from google.colab import userdata
from transformers import pipeline
import json
import re

### **Acceder a gemini**

In [ ]:
client = OpenAI(
    api_key=userdata.get('GOOGLE_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

texto = "Alberto compró 3 tenis por 5000 pesos el 27 de Junio en Dportenis."

## **Function Calling**

**Function Calling** permite invocar funciones externas definidas por el usuario cuando detectan que la intención del mensaje requiere ejecutar una acción específica, en lugar de solo generar texto libre.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "extraccion_compra",
            "description": "Extrae información de una compra desde una frase",
            "parameters": {
                "type": "object",
                "properties": {
                    "cliente": { "type": "string" },
                    "producto": {
                        "type": "array",
                        "description": "Lista de productos adquiridos",
                        "items": {
                            "type": "object",
                            "properties": {
                                "nombre": { "type": "string" },
                                "cantidad": { "type": "integer" },
                                "precio_total": { "type": "number" }
                            },
                            "required": ["nombre", "cantidad", "precio_total"]
                        }
                    },
                    "fecha": { "type": "string" },
                    "lugar": { "type": "string" }
                },
                "required": ["cliente", "producto", "fecha", "lugar"]
            }
        }
    }
]

messages = [
    {"role": "user", "content": texto}
]

response = client.chat.completions.create(
    model="models/gemini-2.0-flash",
    messages=messages,
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "extraccion_compra"}}
)

tool_response = response.choices[0].message.tool_calls[0].function.arguments

print(tool_response)

## **Outlines**

"**Outlines**" son una representación estructurada y jerárquica de los elementos clave de un texto, que resume su contenido organizándolo en secciones, subtítulos, y/o categorías semánticas (como quién, qué, cuándo, dónde).

In [ ]:
ner_pipeline = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", grouped_entities=True)
entidades = ner_pipeline(texto)
print(entidades)

In [ ]:
outline = {
    "cliente": None,
    "detalles": {
        "productos": [],
        "precio_total": None,
        "fecha_compra": None,
        "lugar_compra": None
    }
}

for entidad in entidades:
    entidad_text = entidad['word'].strip()
    entidad_type = entidad['entity_group']

    if entidad_type == 'PER':
        outline['cliente'] = entidad_text
    elif entidad_type == 'LOC':
        outline["detalles"]['lugar_compra'] = entidad_text


producto_match = re.search(r"(\d+)\s+(\w+)\s+(\w+)\s+(\w+)\s+(\w+)", texto)

if producto_match:
    cantidad = int(producto_match.group(1))
    tipo = producto_match.group(2)

    producto = {
        "tipo": tipo,
        "cantidad": cantidad,
        "precio_unitario": None
    }

    outline['detalles']['productos'].append(producto)

precio = re.search(r"\b(\d+)\s*(pesos|mxn|USD|dólares?)\b", texto, re.IGNORECASE)
if precio:
    total = int(precio.group(1))
    outline["detalles"]["precio_total"] = total

    if outline["detalles"]["productos"]:
        outline["detalles"]["productos"][0]["precio_unitario"] = total // cantidad

fecha = re.search(r"el\s([\d]{1,2}\s+de\s+\w+)", texto, re.IGNORECASE)
if fecha:
    outline["detalles"]["fecha_compra"] = fecha.group(1)

output_json = json.dumps(outline, indent=2, ensure_ascii=False)
print(output_json)

## **Ejercicio 02**

Sigue las siguientes instrucciones para generar un JSON puro utilizando la técnica **Function Calling**.

**Texto**: El 27 de junio, Carlos Méndez visitó la sucursal de Dportenis Las Torres y realizó una compra especial. Adquirió 2 pares de tenis Puma Future Z, valorados en 1800 pesos cada uno, ideales para sus entrenamientos de fútbol. Además, compró una playera deportiva Under Armour DryFit por 900 pesos, perfecta para sus sesiones de gimnasio. En total, la compra ascendió a 4500 pesos.

**Estructura de respuesta esperada**:
```
{
  "cliente": "string",
  "detalles_compra": {
    "productos": [
      {
        "tipo": "string",
        "marca": "string",
        "modelo": "string",
        "cantidad": "number",
        "precio_unitario": "number"
      }
    ],
    "precio_total": "number",
    "fecha_compra": "string",
    "lugar_compra": "string"
  }
}
```


In [ ]:
#Escribe el texto de la instrucción
texto = ""

#Aplica una de las tecnicas anteriores...
